In [1]:
# -*- coding:utf-8 -*-
import json
import requests
import csv
import re
from lxml import etree
from datetime import date,datetime
import pandas as pd
import math
from copy import deepcopy

In [2]:
#Save_interest_rate

In [3]:
def save_interest_rate(data):
    
    # 1. 创建文件对象
    f = open('CB_interest_rate.csv', 'w', encoding='utf-8', newline='')
    # 2. 基于文件对象构建 csv写入对象
    csv_writer = csv.writer(f)
    # 3. 构建列表头
    csv_writer.writerow(["id","iry1", "iry2", "iry3", "iry4", "iry5", "iry6"])
    
    count = 0
    for item in data:
        count +=1
        if (count %10 == 0):
            print("processing "+str(count)+" items")
        try:
            interest = get_interest_rate(item['id'])
            interest = [item['id']]+interest
            csv_writer.writerow(interest)
        except Exception as e:
            print("Oops!", e.__class__, "occurred.")
            print("The error happed at "+item['id'])
    # 5. 关闭文件
    print("Finished processing "+str(count)+" items")
    f.close()


In [4]:
#append_interest_rate

In [5]:
def append_interest_rate(item):
    
    # 1. 创建文件对象
    interest_dict = get_interest_dict('CB_interest_rate.csv')
    if item['id'] in interest_dict:
        return
    f = open('CB_interest_rate.csv', 'a', encoding='utf-8', newline='')
    # 2. 基于文件对象构建 csv写入对象
    csv_writer = csv.writer(f)
    
    

    try:
        interest = get_interest_rate(item['id'])
        interest = [item['id']]+interest
        csv_writer.writerow(interest)
        print("Finished appending new CB "+ item['name']+" into interest rate files")
    except Exception as e:
        print("Oops!", e.__class__, "occurred.")
        print("The error happed at "+item['id'])
    
    # 5. 关闭文件
    
    f.close()


In [6]:
#get_interest_rate

In [7]:
def get_interest_rate(id):
    years = 6
    detials_url = 'https://www.jisilu.cn/data/convert_bond_detail/'+id
    detials_response = requests.get(detials_url)
    html = detials_response.content.decode("utf-8")
    html = etree.HTML(html)
    interest = html.xpath('.//td[@id="cpn_desc"]/text()')
    pattern = re.compile(r'(?<!\d|\.)\d+(?:\.\d+)?')  #get proper format number
    interest = pattern.findall(interest[0])
    
    if(len(interest) == 1):
        interest = interest*years
    return interest

In [8]:
# get interest dict

In [9]:
def get_interest_dict(file):
    interest_dict = dict()
    with open(file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                #skip header
                line_count += 1
            else:
                interest_dict[row[0]] = row[1:]
                line_count += 1
        print(f'Processed {line_count} lines.')
    return interest_dict


In [10]:
# buy or sell cb

In [11]:
def buy_sell_db(db_df,file):
    keep_number = 20
    add_number = 5
    keep_dict = {i:db_df['name'][i] for i in db_df[0:keep_number].index}
    add_dict = {i:db_df['name'][i] for i in db_df[0:add_number].index}
    sell_count = 0
    holding_dict = dict()
    
    ##sell
    with open(file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            holding_dict[row[0]] = row[1]
            if row[0] not in keep_dict.keys():
                print(f'Sell {row[0]} {row[1]}')
                sell_count += 1
    if(sell_count==0):
        print('No CB to sell')
        
    ##buy
    print()
    for cb_id,name in add_dict.items():
        print("Add  "+ cb_id+' '+name)
    print()
    for cb_id in keep_dict.keys():
        if cb_id not in holding_dict.keys():
            print("Buy  "+cb_id+' '+keep_dict[cb_id])
    

In [12]:
# get data

In [13]:
def get_dat(get_force_redeem="No"):
    headers = {
        #"User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36",
        "User-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36 OPR/38.0.2220.41",
    }

    newUrl ="https://www.jisilu.cn/data/cbnew/cb_list/?___jsl=LST___t=1584777951900"
    #最简单的爬虫请求.也可以加上headers字段，防止部分网址的反爬虫机制
    response = requests.get(newUrl)
    #当爬取的界面需要用户名密码登录时候，构建的请求需要包含auth字段
    data = response.content.decode("utf-8")
    dat = json.loads(data)

    # 所有数据
    lst_data = []
    count = 0
    for one in dat['rows']:
        # 每一条数据
        lst_dat = dict()
        # 转债id
        id = one["id"]
        dat_cell = one["cell"]
        # 是否赎回
        is_force_redeem = dat_cell['force_redeem']
        is_list = dat_cell['price_tips']
        
            
        
        force_redeem_boolean = False
        if(get_force_redeem=="No"):
            if(is_force_redeem == None):
                force_redeem_boolean = True
            else:
                force_redeem_boolean = False
        else:
            if(is_force_redeem == None):
                force_redeem_boolean = False
            else:
                force_redeem_boolean = True
        
        if force_redeem_boolean and is_list != "待上市":
            
            count += 1
            if (count %100 == 0):
                print("processing "+str(count)+" items")
            
            # 转债名称
            name = dat_cell['bond_nm']
            # 现价
            price = dat_cell['price']
            # stock id
            stock_id = dat_cell['stock_id']
            stock_id = stock_id.replace("sh", "ss")
            stock_id = stock_id[2:]+"."+stock_id[0:2]
            # stock name
            stock_nm = dat_cell['stock_nm']
            # 溢价率
            premium_rt = dat_cell['premium_rt']
            # 评级
            rating_cd = dat_cell['rating_cd']
            
            
            # next put date
            next_put_dt = dat_cell['next_put_dt']
            # put price
            put_price   = dat_cell['put_price']
            # 回售触发价
            put_convert_price = dat_cell['put_convert_price']
            
            
            # maturity date
            maturity_dt = dat_cell['maturity_dt']
            # redeem price
            redeem_price = dat_cell['redeem_price']
            
            # 强赎触发价
            redeem_dt = dat_cell['redeem_dt']
            force_redeem_price = dat_cell['force_redeem_price']
            # 剩余时间
            last_time = dat_cell['year_left']
            pb = dat_cell['pb']


            #lst_dat.append(id)
            lst_dat['id'] = id
            lst_dat['name'] = name
            lst_dat['stock_id'] = stock_id
            lst_dat['stock_name'] = stock_nm
            lst_dat['price'] = price
            lst_dat['convert_value'] = dat_cell['convert_value']
            lst_dat['premium_rt'] = premium_rt
            lst_dat['rating_cd'] = rating_cd
            lst_dat['pb']  = pb
            lst_dat['convert_amt_ratio'] = dat_cell['convert_amt_ratio']
            lst_dat['convert_amt_ratio2'] = dat_cell['convert_amt_ratio2']
            
            
            
            if(get_force_redeem == "No"):
                lst_dat['next_put_dt'] = next_put_dt
                lst_dat['put_price'] = put_price
                lst_dat['put_convert_price'] = put_convert_price
            
                lst_dat['maturity_dt'] = maturity_dt
                lst_dat['redeem_price'] = redeem_price
                lst_dat['last_time'] = last_time
            
            else:
                lst_dat['redeem_dt']  = redeem_dt
                lst_dat['force_redeem_price'] = force_redeem_price
                lst_dat['force_redeem'] = is_force_redeem
            
  
            lst_data.append(lst_dat)
            
        else:
            continue
    print("Finished processing "+str(count)+" items")
    
    return lst_data

In [14]:
#put arbitrage

In [15]:
def put_arbitrage(data):
    data_copy = deepcopy(data)
    put_data = []
    count = 0
    for item in data_copy:
        price       = item['price']
        put_price   = item['put_price']

        
        if(put_price is None):
            continue
            
        price = float(price)
        put_price = float(put_price)
            
        if(price>put_price+5.0): ###To Save Time, the maximun interest could not over 10
            continue
            

        
        next_put_dt = item['next_put_dt']
        next_put_dt = datetime.strptime(next_put_dt,'%Y-%m-%d').date()
        maturity_dt = item['maturity_dt']
        maturity_dt = datetime.strptime(maturity_dt,'%Y-%m-%d').date()
        today = date.today()
        
        
        days_to_maturity = maturity_dt-today
        years_to_maturity = days_to_maturity.days/365
        days_to_put  = next_put_dt-today
        years_to_put  = days_to_put.days/365
        
        put_to_maturity = round(years_to_maturity-years_to_put)
        #print(put_to_maturity)
        
        
        if item['id'] in interest_dict.keys():
            interest = interest_dict[item['id']]
        else:
            append_interest_rate(item)
            interest = get_interest_rate(item['id'])
        
        
        total_interest = 0
        for i in range(3,math.ceil(years_to_put)+put_to_maturity+1):
            #print(i)
            #print(interest)
            #print(item['id'])
            total_interest += float(interest[-i])
        
        
    
        
        actual_put_price = put_price*(1+total_interest*0.008)
        
        
        if(price<actual_put_price):
            year_diff = years_to_put + 38/365
            profit_rate = actual_put_price/price
            CAGR  = profit_rate ** (1/year_diff)-1
            item['actual_put_price'] = actual_put_price
            item['CAGR'] = CAGR
            item['last_to_put'] = year_diff
            put_data.append(item)
        
        count += 1
        if (count %100 == 0):
            print("processing"+str(count)+" items")
    
    print("Finished processing "+str(count)+" items")
    df = pd.DataFrame(put_data)
    df = df.set_index(['id'])
    df = df.sort_values('CAGR',ascending=False)
    return df
          

In [16]:
# redeem arbitrage

In [17]:
def redeem_arbitrage(data):
    put_data = []
    data_copy = deepcopy(data)
    for item in data_copy:
        price       = item['price']
        redeem_price   = item['redeem_price']

        
        if(redeem_price is None):
            continue
            
        price = float(price)
        redeem_price = float(redeem_price)
            

        
        maturity_dt = item['maturity_dt']
        maturity_dt = datetime.strptime(maturity_dt,'%Y-%m-%d').date()
        today = date.today()
        
        days_to_maturity = maturity_dt-today
        years_to_maturity = days_to_maturity.days/365
        
        
        if item['id'] in interest_dict.keys():
            interest = interest_dict[item['id']]
        else:
            append_interest_rate(item)
            interest = get_interest_rate(item['id'])
            
        total_interest = 0
        for i in range(2,math.ceil(years_to_maturity)):
            total_interest += float(interest[-i])
            
        actual_price = redeem_price*(1+total_interest*0.008)
        
        if(price<actual_price):
            year_diff = years_to_maturity
            profit_rate = actual_price/price
            CAGR  = profit_rate ** (1/year_diff)-1
            item['CAGR'] = CAGR
            put_data.append(item)
    df = pd.DataFrame(put_data)
    df = df.set_index(['id'])
    df = df.sort_values('CAGR',ascending=False)
    return df
          

In [18]:
# double low arbitarge

In [19]:
def double_low_arbitrage(data,redeem_df,put_df):
    db_data = []
    DISCOUNT = 0.8  ###convert amt ratio discount 
    data_copy = deepcopy(data)
    
    redeem_dict = redeem_df.to_dict('index')
    put_dict = put_df.to_dict('index')
    
    for item in data_copy:
        id = item['id']
        price       = item['price']
        premium_rt = item['premium_rt']
        convert_amt_ratio = item['convert_amt_ratio']
        convert_value = item['convert_value']
        
            
        price = float(price)
        premium_rt = float(premium_rt.strip('%'))/100
        convert_amt_ratio = float(convert_amt_ratio.strip('%'))/100
        convert_value = float(convert_value)
        
        new_convert_value = convert_value*(1-convert_amt_ratio*DISCOUNT)
        new_premium = price/new_convert_value - 1
        
        redeem_cagr = 0
        put_cagr = 0
        if(id in redeem_dict.keys()):
            redeem_cagr = redeem_dict[id]['CAGR']
        if(id in put_dict.keys()):
            put_cagr = put_dict[id]['CAGR']
        
        item['redeem_CAGR'] = redeem_cagr
        item['put_CAGR']    = put_cagr   
        item['new_premium'] = new_premium
        if(redeem_cagr>put_cagr):
            db_low = price + new_premium*100 - redeem_cagr*100
        else:
            db_low = price + new_premium*100 - put_cagr*100
        
        item['db_low'] = db_low*math.sqrt(price/100) if price>100 else db_low
        
        db_data.append(item)
    df = pd.DataFrame(db_data)
    df = df.set_index(['id'])
    df = df.sort_values('db_low')
    return df

In [20]:
data = get_dat()
pd.DataFrame.from_dict(data).to_csv(r'CB_list.csv',index=False)
#force_redeem_data = get_dat('Yes')
#pd.DataFrame.from_dict(force_redeem_data).to_csv(r'CB_force_redeem_list.csv',index=False)

processing 100 items
processing 200 items
processing 300 items
Finished processing 323 items


In [21]:
interest_dict = get_interest_dict('CB_interest_rate.csv')

Processed 338 lines.


In [22]:
redeem_df = redeem_arbitrage(data)

In [23]:
put_df = put_arbitrage(data)

Finished processing 57 items


In [24]:
db_df = double_low_arbitrage(data,redeem_df,put_df)

In [25]:
db_df[0:10]

,name,stock_id,stock_name,price,convert_value,premium_rt,rating_cd,pb,convert_amt_ratio,convert_amt_ratio2,next_put_dt,put_price,put_convert_price,maturity_dt,redeem_price,last_time,redeem_CAGR,put_CAGR,new_premium,db_low
id,,,,,,,,,,,,,,,,,,,,
128085,鸿达转债,002002.sz,鸿达兴业,115.200,114.83,0.32%,B,1.56,5.7%,5.7%,2023-12-18,100.000,2.74,2025-12-16,118.000,4.405,0.015871,0.000000,0.051155,127.432665
128087,孚日转债,002083.sz,孚日股份,103.750,94.89,9.34%,AA-,1.07,16.8%,16.8%,2023-12-18,100.000,3.15,2025-12-17,108.000,4.408,0.016921,0.000000,0.263137,130.756374
110051,中天转债,600522.ss,中天科技,104.140,91.89,13.33%,AA+,1.17,14.1%,14.1%,2023-02-28,100.000,6.99,2025-02-28,109.000,3.608,0.020061,0.000000,0.277403,132.535253
110072,广汇转债,600297.ss,广汇汽车,92.510,70.47,31.27%,AA+,0.56,14.6%,14.6%,2024-08-19,100.000,2.82,2026-08-17,110.000,5.074,0.041637,0.031658,0.486365,136.982759
132018,G三峡EB1,600900.ss,长江电力,119.300,112.54,6.13%,AAA,2.45,4.1%,4.1%,2023-04-10,100.000,11.72,2024-04-09,108.000,2.718,0.000000,0.000000,0.096017,140.792276
128071,合兴转债,002228.sz,合兴包装,107.030,86.12,24.27%,AA,1.27,6.7%,6.7%,2023-08-15,100.000,2.93,2025-08-15,110.000,4.068,0.015154,0.000000,0.313188,141.561336
113039,嘉泽转债,601619.ss,嘉泽新能,120.240,115.32,4.27%,AA,2.47,5.9%,5.7%,2024-08-23,100.000,2.42,2026-08-23,108.000,5.090,0.000000,0.000000,0.094316,142.190053
128090,汽模转2,002510.sz,天汽模,111.800,96.69,15.63%,AA-,1.84,8.1%,8.1%,2023-12-27,100.000,2.96,2025-12-27,110.000,4.436,0.003974,0.000000,0.236391,142.787060
113610,灵康转债,603669.ss,灵康药业,122.350,121.37,0.81%,AA-,4.92,7.0%,7.0%,2024-11-29,100.000,6.03,2026-11-30,115.000,5.362,0.000000,0.000000,0.067876,142.841543


In [26]:
db_df[10:20]

,name,stock_id,stock_name,price,convert_value,premium_rt,rating_cd,pb,convert_amt_ratio,convert_amt_ratio2,next_put_dt,put_price,put_convert_price,maturity_dt,redeem_price,last_time,redeem_CAGR,put_CAGR,new_premium,db_low
id,,,,,,,,,,,,,,,,,,,,
113009,广汽转债,601238.ss,广汽集团,118.690,107.37,10.54%,AAA,1.79,2.4%,1.6%,2021-01-22,100.680,9.78,2022-01-22,106.000,0.504,0.000000,0.0,0.127070,143.150335
123087,明电转债,300739.sz,明阳电路,112.300,102.27,9.81%,AA-,3.10,14.8%,14.4%,2025-12-15,100.000,9.79,2026-12-14,115.000,5.400,0.012881,0.0,0.245546,143.662147
110071,湖盐转债,600929.ss,雪天盐业,116.160,108.28,7.28%,AA,2.38,11.1%,10.9%,2024-07-09,100.000,4.56,2026-07-09,110.000,4.967,0.000000,0.0,0.177320,144.305552
113030,东风转债,601515.ss,东风股份,113.210,94.57,19.71%,AA,1.75,3.6%,3.6%,2023-12-25,100.000,4.51,2025-12-23,112.000,4.425,0.005229,0.0,0.232602,144.648148
128037,岩土转债,002542.sz,中化岩土,107.980,92.58,16.63%,AA,1.26,14.5%,11.6%,2022-03-15,100.000,2.17,2024-03-15,108.000,2.649,0.005481,0.0,0.319392,144.825246
110041,蒙电转债,600863.ss,内蒙华电,107.180,90.62,18.27%,AAA,0.90,13.9%,13.9%,2021-12-21,100.000,1.79,2023-12-21,106.000,2.416,0.001336,0.0,0.330717,145.061110
128081,海亮转债,002203.sz,海亮股份,117.490,113.73,3.31%,AA,2.14,14.7%,13.9%,2023-11-21,100.000,6.78,2025-11-21,110.000,4.337,0.000000,0.0,0.170740,145.857666
123002,国祯转债,300388.sz,节能国祯,110.000,88.64,24.10%,AA,1.39,4.9%,4.9%,2021-11-24,100.000,5.85,2023-11-24,106.000,2.342,0.000000,0.0,0.291606,145.952834
123096,思创转债,300078.sz,思创医惠,112.638,98.43,14.44%,AA-,2.21,12.0%,11.6%,2025-01-27,100.000,5.78,2027-01-25,118.000,5.515,0.016514,0.0,0.265870,146.008299


In [27]:
buy_sell_db(db_df,'mycb_list.csv')

Sell 113017 吉视转债
Sell 128100 搜特转债

Add  128085 鸿达转债
Add  128087 孚日转债
Add  110051 中天转债
Add  110072 广汇转债
Add  132018 G三峡EB1

Buy  132018 G三峡EB1
Buy  128090 汽模转2
Buy  113565 宏辉转债
Buy  113030 东风转债
Buy  128037 岩土转债
Buy  110041 蒙电转债
Buy  128081 海亮转债
Buy  123002 国祯转债
Buy  123096 思创转债
Buy  110033 国贸转债


In [28]:
put_df[0:10]

,name,stock_id,stock_name,price,convert_value,premium_rt,rating_cd,pb,convert_amt_ratio,convert_amt_ratio2,next_put_dt,put_price,put_convert_price,maturity_dt,redeem_price,last_time,actual_put_price,CAGR,last_to_put
id,,,,,,,,,,,,,,,,,,,
128062,亚药转债,002370.sz,亚太药业,79.851,28.62,179.05%,BB,3.92,44.7%,38.6%,2023-04-03,100.000,11.38,2025-04-02,115.000,3.699,102.00,0.145452,1.802740
128100,搜特转债,002503.sz,搜于特,76.065,48.97,55.34%,A,1.26,22.1%,18.2%,2024-03-11,100.000,2.03,2026-03-11,112.000,4.638,102.48,0.114817,2.742466
113595,花王转债,603007.ss,ST花王,85.550,48.41,76.72%,A,0.96,29.5%,29.3%,2024-07-22,100.000,4.84,2026-07-20,116.000,4.997,103.04,0.061702,3.106849
128044,岭南转债,002717.sz,岭南股份,96.666,49.75,94.32%,AA-,1.00,19.9%,14.7%,2022-08-14,100.000,4.14,2024-08-14,107.000,3.066,102.00,0.047095,1.167123
113017,吉视转债,601929.ss,吉视传媒,98.860,61.36,61.13%,AA+,0.81,27.7%,27.7%,2021-12-27,100.000,2.06,2023-12-27,106.000,2.433,101.04,0.041455,0.536986
113596,城地转债,603887.ss,城地香江,91.910,39.61,132.02%,AA-,1.05,35.3%,27.7%,2024-07-29,100.000,16.98,2026-07-27,108.000,5.016,102.80,0.036470,3.126027
127019,国城转债,000688.sz,国城矿业,92.565,44.82,106.51%,AA,4.35,7.9%,7.9%,2024-07-15,100.000,14.74,2026-07-14,110.000,4.981,102.40,0.033243,3.087671
110072,广汇转债,600297.ss,广汇汽车,92.510,70.47,31.27%,AA+,0.56,14.6%,14.6%,2024-08-19,100.000,2.82,2026-08-17,110.000,5.074,102.16,0.031658,3.183562
113589,天创转债,603608.ss,天创时尚,95.070,50.20,89.37%,AA,1.53,22.7%,22.7%,2024-06-24,100.000,8.60,2026-06-23,110.000,4.923,102.48,0.025079,3.030137
